### Train a pos_tag model
- Using three datasets (Airbnb, Twitter, Yelp) <br>
- Keeps words and pos tags into a dictionary, where the key is a word and the value is a counter of POS and counts <br>
- For each word, only check if the most common pos_tag is equal. <br>

In [9]:
from nltk import pos_tag
import pandas as pd
import pickle, re
from sklearn.feature_extraction.text import CountVectorizer
from collections import defaultdict, Counter

In [5]:
airbnb_path = "/data/2/zwang/2018_S_WordTreatment/V2_airbnb/"
tw_path = "/data/2/zwang/2018_S_WordTreatment/V2_twitter/"
yp_path = "/data/2/zwang/2018_S_WordTreatment/V2_yelp/"

In [2]:
airbnb_corpus = "/data/2/zwang/2018_S_WordTreatment/V2_airbnb/1_Process/airbnb_labelby_sents_intotal.pickle"
tw_corpus = "/data/2/zwang/2018_S_WordTreatment/V2_twitter/1_Process/tw_labelby_sents_intotal.pickle"
yp_corpus = "/data/2/zwang/2018_S_WordTreatment/V2_yelp/1_Process/yp_labelby_sents_intotal.pickle"

In [36]:
def count_tags(airbnb_corpus,yp_corpus,tw_corpus,tag_file):
    word_tags = defaultdict(Counter)
    airbnb_pd = pd.DataFrame(pickle.load(open(airbnb_corpus,'rb')))
    yp_pd = pd.DataFrame(pickle.load(open(yp_corpus,'rb')))
    tw_pd = pd.DataFrame(pickle.load(open(tw_corpus,'rb')))
    
    for data_pd in [airbnb_pd,yp_pd,tw_pd]:
        print(data_pd.shape)
        for idx, row in data_pd.iterrows():
            if(idx % 10000 ==0):
                print(idx)
            tag_sent = pos_tag(row.sentence.split())
            for item in tag_sent:
                word_tags[item[0]][item[1]] += 1
    
    #pickle.dump(word_tags,open(tag_file,'wb'))
    return word_tags

In [37]:
#word_tags = count_tags(airbnb_corpus,yp_corpus,tw_corpus,tag_file='/data/2/zwang/2018_S_WordTreatment/V2_airbnb/word_tag_counter.pickle')

(358750, 2)
0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
(1369991, 2)
0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000
790000
800000
810000
820000
830000
840000
850000
860000
870000
880000
890000
900000
910000
920000
930000
940000
950000
960000
970000
980000
990000
1000000
1010000
1020000
1030000
1040000
1050000
106

In [4]:
word_tags = pickle.load(open('/data/2/zwang/2018_S_WordTreatment/V2_airbnb/word_tag_counter.pickle','rb'))

In [50]:
word_tags['skinny']

Counter({'CC': 1,
         'JJ': 534,
         'JJR': 1,
         'NN': 172,
         'NNS': 3,
         'RB': 6,
         'VB': 21,
         'VBD': 3,
         'VBN': 12,
         'VBP': 11,
         'VBZ': 3})

In [51]:
word_tags['limited']

Counter({'JJ': 1278, 'VB': 2, 'VBD': 80, 'VBN': 560})

In [62]:
def store_word_tags(word_tags,treat_pairs_file,wd_tags_file):
    pair_pd = pd.read_csv(treat_pairs_file)
    vocab = list(set(pair_pd.source.values))+list(set(pair_pd.target.values))
    wd_tags = defaultdict(Counter)
    for wd in vocab:
        for tag_ct in word_tags[wd]:
            if(word_tags[wd][tag_ct]>20):
                wd_tags[wd][tag_ct] = word_tags[wd][tag_ct]
    
    pickle.dump(wd_tags,open(wd_tags_file,'wb'))
    return wd_tags

In [72]:
wd_tags = store_word_tags(word_tags,
                treat_pairs_file = '/data/2/zwang/2018_S_WordTreatment/V2_airbnb/1_Process/treat_pairs.csv',
                wd_tags_file = '/data/2/zwang/2018_S_WordTreatment/V2_airbnb/1_Process/wd_tags.pickle')

In [48]:
word_tags['canvas']

Counter({'JJ': 4, 'NN': 85, 'VB': 1})

In [49]:
word_tags['display']

Counter({'NN': 681, 'VB': 51, 'VBD': 2, 'VBN': 1, 'VBP': 40})

### Check if the most common pos_tag is equal

In [180]:
def check_com_pos(wd_tags_file,word_pairs_file,remain_wdpairs_file):
    wd_tags_pd = pickle.load(open(wd_tags_file,'rb'))
    wd_pairs_pd = pd.read_csv(word_pairs_file)
    remain_idx = []
    for idx,row in wd_pairs_pd.iterrows():
        if((row.source in wd_tags_pd) and (row.target in wd_tags_pd)):
            if(wd_tags_pd[row.source].most_common(1)[0][0] == wd_tags_pd[row.target].most_common(1)[0][0]):
                remain_idx.append(idx)
            else:
                print(row.source,wd_tags_pd[row.source].most_common(1),row.target,wd_tags_pd[row.target].most_common(1))
    
    wd_pairs_pd.iloc[remain_idx].to_csv(remain_wdpairs_file)
    print("%d remained, and %d removed" % (len(remain_idx),wd_pairs_pd.shape[0]-len(remain_idx)))

In [181]:
wd_tags_file = '/data/2/zwang/2018_S_WordTreatment/V2_airbnb/1_Process/wd_tags.pickle'
word_pairs_file = '/data/2/zwang/2018_S_WordTreatment/V2_airbnb/1_Process/treat_pairs_posinters.csv'
remain_wdpairs_file = '/data/2/zwang/2018_S_WordTreatment/V2_airbnb/1_Process/treat_pairs_posinters_poscom1.csv'
check_com_pos(wd_tags_file,word_pairs_file,remain_wdpairs_file)

best [('JJS', 52021)] well [('RB', 37021)]
best [('JJS', 52021)] good [('JJ', 114386)]
shops [('NNS', 10731)] store [('NN', 13779)]
close [('RB', 12616)] closest [('JJS', 993)]
find [('VB', 20023)] found [('VBD', 9152)]
near [('IN', 7910)] closest [('JJS', 993)]
2nd [('CD', 2186)] second [('JJ', 7428)]
high [('JJ', 11680)] highest [('JJS', 713)]
best [('JJS', 52021)] nice [('JJ', 37603)]
goods [('NNS', 836)] nice [('JJ', 37603)]
quick [('JJ', 8625)] rapidly [('RB', 272)]
children [('NNS', 2279)] kid [('NN', 3625)]
quick [('JJ', 8625)] fast [('RB', 4676)]
fast [('RB', 4676)] quick [('JJ', 8625)]
artisan [('JJ', 242)] craftsman [('NN', 64)]
vibrant [('JJ', 2384)] lively [('RB', 1511)]
safe [('JJ', 9018)] secure [('NN', 180)]
closer [('JJR', 906)] closest [('JJS', 993)]
best [('JJS', 52021)] better [('JJR', 21439)]
prepared [('VBN', 1790)] ready [('JJ', 9563)]
underground [('JJ', 252)] metro [('NN', 564)]
big [('JJ', 17901)] larger [('JJR', 1301)]
flat [('JJ', 1956)] apartment [('NN', 103

In [182]:
wd_tags_file = '/data/2/zwang/2018_S_WordTreatment/V2_twitter/1_Process/wd_tags.pickle'
word_pairs_file = '/data/2/zwang/2018_S_WordTreatment/V2_twitter/1_Process/treat_pairs_posinters.csv'
remain_wdpairs_file = '/data/2/zwang/2018_S_WordTreatment/V2_twitter/1_Process/treat_pairs_posinters_poscom1.csv'
check_com_pos(wd_tags_file,word_pairs_file,remain_wdpairs_file)

well [('RB', 37021)] good [('JJ', 114386)]
well [('RB', 37021)] best [('JJS', 52021)]
good [('JJ', 114386)] better [('JJR', 21439)]
good [('JJ', 114386)] well [('RB', 37021)]
good [('JJ', 114386)] best [('JJS', 52021)]
meant [('VBN', 556)] mean [('VBP', 4838)]
build [('VB', 714)] built [('VBN', 1308)]
happier [('JJR', 776)] happy [('JJ', 21051)]
cheap [('JJ', 4807)] cheaper [('JJR', 1540)]
win [('VB', 1503)] won [('VBD', 1339)]
child [('NN', 1748)] kids [('NNS', 9962)]
nice [('JJ', 37603)] best [('JJS', 52021)]
nice [('JJ', 37603)] better [('JJR', 21439)]
closing [('NN', 577)] shut [('VBD', 723)]
nice [('JJ', 37603)] goods [('NNS', 836)]
hit [('VBD', 2659)] strike [('NN', 216)]
alright [('VBN', 529)] good [('JJ', 114386)]
limited [('JJ', 1278)] smallest [('JJS', 256)]
kid [('NN', 3625)] children [('NNS', 2279)]
alright [('VBN', 529)] okay [('JJ', 1968)]
enjoyed [('VBD', 5282)] come [('VB', 19437)]
happiest [('JJS', 322)] happy [('JJ', 21051)]
display [('NN', 681)] showing [('VBG', 1130

In [183]:
wd_tags_file = '/data/2/zwang/2018_S_WordTreatment/V2_yelp/1_Process/wd_tags.pickle'
word_pairs_file = '/data/2/zwang/2018_S_WordTreatment/V2_yelp/1_Process/treat_pairs_posinters.csv'
remain_wdpairs_file = '/data/2/zwang/2018_S_WordTreatment/V2_yelp/1_Process/treat_pairs_posinters_poscom1.csv'
check_com_pos(wd_tags_file,word_pairs_file,remain_wdpairs_file)

chose [('VBD', 1637)] choose [('VB', 3280)]
well [('RB', 37021)] good [('JJ', 114386)]
well [('RB', 37021)] best [('JJS', 52021)]
better [('JJR', 21439)] well [('RB', 37021)]
good [('JJ', 114386)] better [('JJR', 21439)]
good [('JJ', 114386)] well [('RB', 37021)]
good [('JJ', 114386)] best [('JJS', 52021)]
delighted [('VBN', 164)] pleased [('JJ', 1435)]
start [('VB', 7529)] beginning [('NN', 969)]
happier [('JJR', 776)] happy [('JJ', 21051)]
better [('JJR', 21439)] decent [('JJ', 6734)]
strike [('NN', 216)] hit [('VBD', 2659)]
alright [('VBN', 529)] good [('JJ', 114386)]
cold [('JJ', 5435)] colder [('NN', 79)]
alright [('VBN', 529)] okay [('JJ', 1968)]
insanely [('RB', 274)] super [('JJ', 10324)]
good [('JJ', 114386)] alright [('VBN', 529)]
takes [('VBZ', 5075)] pick [('VB', 4180)]
insane [('NN', 549)] mad [('JJ', 3068)]
outdated [('VBN', 118)] stale [('JJ', 458)]
better [('JJR', 21439)] best [('JJS', 52021)]
typical [('JJ', 2505)] representative [('NN', 133)]
choice [('NN', 4822)] cho

### Train bigram model using all corpus data
- Keeps words and containing bigrams into a dictionary, where the key is a word and the value is a counter of bigrams and counts

In [77]:
def train_bigram(airbnb_corpus,yp_corpus,tw_corpus,bigram_file):
    word_bigram = {}
    airbnb_pd = pd.DataFrame(pickle.load(open(airbnb_corpus,'rb')))
    yp_pd = pd.DataFrame(pickle.load(open(yp_corpus,'rb')))
    tw_pd = pd.DataFrame(pickle.load(open(tw_corpus,'rb')))
    bi_vec = CountVectorizer(ngram_range=(2,2),stop_words=None,token_pattern='[a-z]+')
    
    for data_pd in [airbnb_pd,yp_pd,tw_pd]:
        print(data_pd.shape)
        X_data = bi_vec.fit_transform(data_pd.sentence.values)
        feat_ct = X_data.sum(axis=0).A1
        for feat in bi_vec.vocabulary_:
            if(feat not in word_bigram):
                word_bigram[feat] = 0
            word_bigram[feat] += feat_ct[bi_vec.vocabulary_[feat]]
        
    pickle.dump(word_bigram,open(bigram_file,'wb'))
    #return word_bigram

In [78]:
train_bigram(airbnb_corpus,yp_corpus,tw_corpus,bigram_file = '/data/2/zwang/2018_S_WordTreatment/V2_airbnb/bigram_dict.pickle')

(358750, 2)
(1369991, 2)
(1341550, 2)


In [81]:
word_bigram = pickle.load(open('/data/2/zwang/2018_S_WordTreatment/V2_airbnb/bigram_dict.pickle','rb'))

In [82]:
len(word_bigram)

4210471

In [83]:
list(word_bigram.keys())[:3]

['mama playing', 'asshole ass', 'flavorful tomatoes']

In [89]:
word_bigram['see movies']

15

### Get bigram info for all word in the paraphrase vocabulary
- Keeps words and its bigrams into a dictionary, where the key is a word and the value is a counter of all containing bigrams and counts

In [96]:
def get_wd_bigram(all_bigram_file,wd_bigram_file):
    all_bigram = pickle.load(open(all_bigram_file,'rb'))
    wd_bigram = defaultdict(Counter)
    for bigram in all_bigram:
        wd1,wd2 = bigram.split()
        wd_bigram[wd1][bigram] += all_bigram[bigram]
        wd_bigram[wd2][bigram] += all_bigram[bigram]
    
    pickle.dump(wd_bigram,open(wd_bigram_file,'wb'))

In [97]:
all_bigram_file = '/data/2/zwang/2018_S_WordTreatment/V2_airbnb/bigram_dict.pickle'
wd_bigram_file = '/data/2/zwang/2018_S_WordTreatment/V2_airbnb/wd_bigram_dfdict.pickle'
get_wd_bigram(all_bigram_file,wd_bigram_file)

In [101]:
wd_bigram_dfdict = pickle.load(open(wd_bigram_file,'rb'))

In [117]:
type(wd_bigram_dfdict)

collections.defaultdict

In [105]:
for item in wd_bigram_dfdict['otherwise']:
    print(item)
    break

vomit otherwise


In [127]:
def get_limitvocab_bigram(wd_bigram_file,airbnb_wdpair_file,tw_wdpair_file,yp_wdpair_file,vocab_bigram_file):
    wd_bigram_dfdict = pickle.load(open(wd_bigram_file,'rb'))
    vocab_ct = Counter()
    airbnb_pd = pd.read_csv(airbnb_wdpair_file)
    vocab_ct.update(list(airbnb_pd.source.values) + list(airbnb_pd.target.values))
    tw_pd = pd.read_csv(tw_wdpair_file)
    vocab_ct.update(list(tw_pd.source.values) + list(tw_pd.target.values))
    yp_pd = pd.read_csv(yp_wdpair_file)
    vocab_ct.update(list(yp_pd.source.values) + list(yp_pd.target.values))
    
    vocab_bigram = defaultdict(Counter)
    for wd in vocab_ct:
        for wd_bi in wd_bigram_dfdict[wd]:
            vocab_bigram[wd][wd_bi] += wd_bigram_dfdict[wd][wd_bi]
    
    pickle.dump(vocab_bigram,open(vocab_bigram_file,'wb'))
    print("Store bigram information for %d words." % len(vocab_ct))

In [128]:
wd_bigram_file = '/data/2/zwang/2018_S_WordTreatment/V2_airbnb/wd_bigram_dfdict.pickle'
airbnb_wdpair_file = '/data/2/zwang/2018_S_WordTreatment/V2_airbnb/1_Process/treat_pairs_posinters.csv'
tw_wdpair_file = '/data/2/zwang/2018_S_WordTreatment/V2_twitter/1_Process/treat_pairs_posinters.csv'
yp_wdpair_file = '/data/2/zwang/2018_S_WordTreatment/V2_yelp/1_Process/treat_pairs_posinters.csv'
vocab_bigram_file = '/data/2/zwang/2018_S_WordTreatment/V2_airbnb/vocab_bigram_dfdict.pickle'
get_limitvocab_bigram(wd_bigram_file,airbnb_wdpair_file,tw_wdpair_file,yp_wdpair_file,vocab_bigram_file)

Store bigram information for 1334 words.


- For a word pair, check all its containing sentences, and select those can be substituted with target word <br>
- check if after substitution by target word, the bigram exists in database <br>
- only maintain sentences that could be substituted by target word, and then divide into 10 folds to randomly select <br>

In [134]:
tw_path = '/data/2/zwang/2018_S_WordTreatment/V2_twitter/'
sort_file = tw_path+'3_CausalForest/tw_csf_200tree_treatment_posinters_sort.pickle'
csf_pd = pd.DataFrame(pickle.load(open(sort_file,'rb')))
csf_pd.shape

(2227554, 6)

In [135]:
csf_pd.head()

,effect,sentence,source,target,true_y,wdpair_sentence_id
1426624,1.30984,Morning time is so peaceful . It's like all th...,peaceful,ping,1,24
1426675,0.96201,This storm is so peaceful right now .,peaceful,ping,0,20
1426681,0.92405,i love mornings because everyone is asleep and...,peaceful,ping,0,26
1426734,0.89409,I love going on walks this late . #sowarm #pea...,peaceful,ping,0,79
1426752,0.85647,this weather is so peaceful i luv it,peaceful,ping,0,97


In [163]:
csf_pd.ix[[1426624,1426675,1426681]]

,effect,sentence,source,target,true_y,wdpair_sentence_id
1426624,1.30984,Morning time is so peaceful . It's like all th...,peaceful,ping,1,24
1426675,0.96201,This storm is so peaceful right now .,peaceful,ping,0,20
1426681,0.92405,i love mornings because everyone is asleep and...,peaceful,ping,0,26


In [6]:
def check_bigram_match(wdpair_sentence_file,word_pair_file,vocab_bigram_file,result_file):
    """
    Iterate over each word pair, and each sentence:
        check if the bigram exists after target word substitution:
        left_word control_word right_word --> left_word treatment_word right_word
        Only remain sentences that either (left_word treatment_word) or (treatment_word right_word) exist in treatment_word's history bigrams.
    return remaining word pairs, number of remaining sentences, and write sentences for each word pairs.
    """
    
    wdpair_pd = pd.DataFrame(pickle.load(open(wdpair_sentence_file,'rb')))
    wd_pairs = pd.read_csv(word_pair_file)
    vocab_bigram_dfdict = pickle.load(open(vocab_bigram_file,'rb'))
    remain_idx = []
    
    for idx,row in wdpair_pd.iterrows():
        # check if this word pair still exists in the limited vocabulary
        if(wd_pairs[(wd_pairs.source==row.source) & (wd_pairs.target==row.target)].shape[0]>0):
            word_list = re.findall('[a-z]+',row.sentence.lower())
            if(row.source in word_list):
                srci = word_list.index(row.source)
                tar_bigram_l = ''
                tar_bigram_r = ''
                if(srci > 0):
                    tar_bigram_l = word_list[srci-1]+' '+row.target
                if(srci < len(word_list)-1):
                    tar_bigram_r = row.target+' '+word_list[srci+1]

                flag = False
                if(tar_bigram_l in vocab_bigram_dfdict[row.target]):
                    flag = True
                elif(tar_bigram_r in vocab_bigram_dfdict[row.target]):
                    flag = True

                if(flag):
                    remain_idx.append(idx)

    print("%d remained out of %d in total" % (len(remain_idx),wdpair_pd.shape[0]))
    
    pickle.dump(wdpair_pd.ix[remain_idx],open(result_file,'wb'))

In [7]:
vocab_bigram_file = '/data/2/zwang/2018_S_WordTreatment/V2_airbnb/vocab_bigram_dfdict.pickle'
word_pair_file = tw_path + '1_Process/treat_pairs_posinters_poscom1.csv'

- results for same dataset should be the same

In [10]:
wdpair_sentence_file = tw_path+'3_CausalForest/tw_csf_200tree_treatment_posinters_sort.pickle'
result_file = tw_path+'3_CausalForest/tw_csf_200tree_treatment_posinters_sort_bigramcheck.pickle'
check_bigram_match(wdpair_sentence_file,word_pair_file,vocab_bigram_file,result_file)

KeyboardInterrupt: 

In [ ]:
wdpair_sentence_file = tw_path+'3_CounterFactual/tw_ctf_200tree_treatment_posinters_sort.pickle'
result_file = tw_path+'3_CounterFactual/tw_ctf_200tree_treatment_posinters_sort_bigramcheck.pickle'
check_bigram_match(wdpair_sentence_file,word_pair_file,vocab_bigram_file,result_file)

In [ ]:
wdpair_sentence_file = tw_path+'3_VirtualTwins/tw_vt_200tree_treatment_posinters_sort.pickle'
result_file = tw_path+'3_VirtualTwins/tw_vt_200tree_treatment_posinters_sort_bigramcheck.pickle'
check_bigram_match(wdpair_sentence_file,word_pair_file,vocab_bigram_file,result_file)

In [ ]:
wdpair_sentence_file = tw_path+'3_KNN/tw_knn_30_treatment_posinters_sort.pickle'
result_file = tw_path+'3_KNN/tw_csf_knn_30_treatment_posinters_sort_bigramcheck.pickle'
check_bigram_match(wdpair_sentence_file,word_pair_file,vocab_bigram_file,result_file)

In [166]:
def generate_wd_pairs(wdpair_bigram_file,original_wdpair_file,remain_wdpairs_file):
    wd_bigram_pd = pd.DataFrame(pickle.load(open(wdpair_bigram_file,'rb')))
    original_wdpairs = pd.read_csv(original_wdpair_file)
    remain_idx = []
    for idx,row in original_wdpairs.iterrows():
        if(wd_bigram_pd[(wd_bigram_pd.source == row.source) & (wd_bigram_pd.target == row.target)].shape[0]>0):
            remain_idx.append(idx)
        else:
            print(row.source,row.target)
            
    print("%d word pairs removed, %d remained" % (original_wdpairs.shape[0]-len(remain_idx),len(remain_idx)))
    original_wdpairs.iloc[remain_idx].to_csv(remain_wdpairs_file,index=False)

,effect,sentence,source,target,true_y,wdpair_sentence_id
1426624,1.30984,Morning time is so peaceful . It's like all th...,peaceful,ping,1,24
1426681,0.92405,i love mornings because everyone is asleep and...,peaceful,ping,0,26
1883368,0.82233,I legit need to dye my hair . This ombr needs ...,dye,ink,0,83
1883369,0.81560,I need to dye my hair black soon again .,dye,ink,0,84
450107,0.81269,"Set a fire down in my soul , that I can't cont...",contain,display,0,15


In [ ]:
wdpair_bigram_file = airbnb_path+'3_KNN/airbnb_knn_30_treatment_posinters_sort_bigramcheck.pickle'
original_wdpair_file = airbnb_path +'1_Process/treat_pairs_posinters_poscom1.csv'
remain_wdpairs_file = airbnb_path +'1_Process/treat_pairs_posinters_poscom1_bigram.csv'
generate_wd_pairs(wdpair_bigram_file,original_wdpair_file,remain_wdpairs_file)

In [ ]:
wdpair_bigram_file = tw_path+'3_KNN/tw_knn_30_treatment_posinters_sort_bigramcheck.pickle'
original_wdpair_file = tw_path +'1_Process/treat_pairs_posinters_poscom1.csv'
remain_wdpairs_file = tw_path +'1_Process/treat_pairs_posinters_poscom1_bigram.csv'
generate_wd_pairs(wdpair_bigram_file,original_wdpair_file,remain_wdpairs_file)

In [170]:
wdpair_bigram_file = yp_path+'3_KNN/yp_knn_30_treatment_posinters_sort_bigramcheck.pickle'
original_wdpair_file = yp_path +'1_Process/treat_pairs_posinters_poscom1.csv'
remain_wdpairs_file = yp_path +'1_Process/treat_pairs_posinters_poscom1_bigram.csv'
generate_wd_pairs(wdpair_bigram_file,original_wdpair_file,remain_wdpairs_file)

"Set a fire down in my soul , that I can't contain , that I can't control .. I want more of you God ! http://url"

In [150]:
sent = "this weather is so Peaceful i luv it"
word_list = re.findall('[a-z]+',sent.lower())
print(word_list)
word_list.index('peaceful')

['this', 'weather', 'is', 'so', 'peaceful', 'i', 'luv', 'it']


4

### If not, apply google n-gram